# Sequence to Sequence model for english to tamil transilation using rnn encoder and decoder ( at character level)

## dataset is downloaded from http://www.manythings.org/anki/

In [23]:
from __future__ import print_function

from keras.models import Model,load_model
from keras.layers import Input,LSTM,Dense
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=128
epochs=25
latent_dim=256
data_path="/home/santhosh/keras/rnn/rnn_encoder_decoder/data/tam.txt"

## vectorizing the data

In [24]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines:
    line=line.split('\t')
    if(len(line)!=2):
        continue
    input_text,target_text=line
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
input_characters=sorted(list(input_characters))
targer_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_len=max([len(text) for text in input_texts])
max_decoder_seq_len=max([len(text) for text in target_texts])

print('Number of samples:',len(input_texts))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


Number of samples: 180
number of unique input token: 53
number of unique output token: 54
Max Sequence length for inputs: 94
Max Sequence length for outputs: 111


## defining token2index

In [25]:
input_token2index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token2index=dict([(char,i) for i,char in enumerate(target_characters)])

## defing encoder_input,decoder_input and decoder_output

In [26]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_tokens),dtype='float32')

## creating training dataset

In [27]:
target_token2index

{'\t': 15,
 '\n': 34,
 ' ': 40,
 '!': 47,
 '(': 18,
 ')': 33,
 ',': 2,
 '.': 8,
 '0': 6,
 '2': 53,
 '?': 43,
 'C': 4,
 'D': 39,
 'அ': 13,
 'ஆ': 36,
 'இ': 25,
 'உ': 14,
 'ஊ': 52,
 'எ': 31,
 'ஏ': 26,
 'ஒ': 1,
 'ஓ': 9,
 'க': 12,
 'ங': 51,
 'ச': 44,
 'ஜ': 7,
 'ஞ': 30,
 'ட': 23,
 'ண': 49,
 'த': 5,
 'ந': 10,
 'ன': 38,
 'ப': 16,
 'ம': 42,
 'ய': 41,
 'ர': 17,
 'ற': 50,
 'ல': 22,
 'ள': 48,
 'ழ': 45,
 'வ': 20,
 'ஷ': 35,
 'ஸ': 24,
 'ா': 28,
 'ி': 21,
 'ீ': 29,
 'ு': 3,
 'ூ': 46,
 'ெ': 27,
 'ே': 32,
 'ை': 37,
 'ொ': 19,
 'ோ': 11,
 '்': 0}

In [28]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token2index[char]]=1
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token2index[char]]=1
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i,t-1,target_token2index[char]]=1


## Define an input sequence and process it.

In [29]:


encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




## Set up the decoder, using encoder_states as initial state.

In [30]:


decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

In [31]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])

## function to test model

In [32]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token2index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token2index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token2index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


## Run training

In [ ]:

iteration=0
"""
# load weights
print('loading the weights')
model=load_model('char_level.h5')

# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
print("Testing Samples\n"+"-"*50)
for i in range(5):
    index=np.random.randint(len(input_texts))
    encoded_input_sequence=encoder_input_data[index: index + 1]
    output_sequence=decode_sequence(encoded_input_sequence)
    print(input_texts[index],output_sequence)
print("-"*50)
"""
iteration_file="/home/santhosh/keras/rnn/rnn_encoder_decoder/iteration_char_level.txt"
try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    
except:
    print('no file exist')

# checkpoint
filepath="weights_best_char_level.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    print('Iteration:',iteration+1)
    #training
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,callbacks=callbacks_list)
    #prepare sample_data to test 5 samples:
    print("-"*50)
    for i in range(5):
        index=np.random.randint(len(input_texts))
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=decode_sequence(encoded_input_sequence)
        print(input_texts[index],output_sequence)
    print("-"*50)
    # Save model
    file=open(iteration_file,'a')
    file.write('iteration:'+str(iteration+1)+'\n')
    file.close()
    iteration+=1
    model.save('char_level.h5')

file_data, iteration:10
Iteration: 11
Train on 144 samples, validate on 36 samples
Epoch 1/25
144/144 [==============================] - 10s 73ms/step - loss: 1.0362 - acc: 0.2618 - val_loss: 1.5485 - val_acc: 0.6539


/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3_2/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3_2/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/25
144/144 [==============================] - 6s 40ms/step - loss: 0.9953 - acc: 0.7787 - val_loss: 1.4390 - val_acc: 0.0470
Epoch 3/25
144/144 [==============================] - 6s 40ms/step - loss: 0.9179 - acc: 0.0308 - val_loss: 1.3321 - val_acc: 0.6552
Epoch 4/25
144/144 [==============================] - 6s 39ms/step - loss: 0.8333 - acc: 0.7817 - val_loss: 1.3041 - val_acc: 0.6592
Epoch 5/25
144/144 [==============================] - 6s 39ms/step - loss: 0.8126 - acc: 0.7848 - val_loss: 1.2987 - val_acc: 0.6564
Epoch 6/25
144/144 [==============================] - 6s 41ms/step - loss: 0.8070 - acc: 0.7836 - val_loss: 1.2953 - val_acc: 0.6564
Epoch 7/25
144/144 [==============================] - 6s 40ms/step - loss: 0.8023 - acc: 0.7823 - val_loss: 1.2866 - val_acc: 0.6592
Epoch 8/25
144/144 [==============================] - 6s 41ms/step - loss: 0.7965 - acc: 0.7848 - val_loss: 1.2941 - val_acc: 0.6564
Epoch 9/25
144/144 [==============================] - 6s 39ms/step - 

Epoch 4/25
144/144 [==============================] - 3s 20ms/step - loss: 0.6699 - acc: 0.1970 - val_loss: 1.1443 - val_acc: 0.7015
Epoch 5/25
144/144 [==============================] - 3s 19ms/step - loss: 0.6697 - acc: 0.7371 - val_loss: 1.1567 - val_acc: 0.6889
Epoch 6/25
144/144 [==============================] - 3s 20ms/step - loss: 0.6703 - acc: 0.7357 - val_loss: 1.1700 - val_acc: 0.6564
Epoch 7/25
144/144 [==============================] - 3s 23ms/step - loss: 0.6601 - acc: 0.7950 - val_loss: 1.1274 - val_acc: 0.6722
Epoch 8/25
144/144 [==============================] - 3s 20ms/step - loss: 0.6504 - acc: 0.8025 - val_loss: 1.1455 - val_acc: 0.1224
Epoch 9/25
144/144 [==============================] - 3s 24ms/step - loss: 0.6462 - acc: 0.1994 - val_loss: 1.1494 - val_acc: 0.1256
Epoch 10/25
144/144 [==============================] - 3s 24ms/step - loss: 0.6451 - acc: 0.2033 - val_loss: 1.1278 - val_acc: 0.6882
Epoch 11/25
144/144 [==============================] - 3s 23ms/step 

Epoch 9/25
144/144 [==============================] - 3s 23ms/step - loss: 0.5141 - acc: 0.7927 - val_loss: 0.9802 - val_acc: 0.6917
Epoch 10/25
144/144 [==============================] - 4s 27ms/step - loss: 0.5053 - acc: 0.7426 - val_loss: 0.9772 - val_acc: 0.6161
Epoch 11/25
144/144 [==============================] - 4s 29ms/step - loss: 0.4949 - acc: 0.7616 - val_loss: 0.9972 - val_acc: 0.1361
Epoch 12/25
144/144 [==============================] - 4s 31ms/step - loss: 0.5078 - acc: 0.1918 - val_loss: 0.9740 - val_acc: 0.1419
Epoch 13/25
144/144 [==============================] - 4s 25ms/step - loss: 0.4977 - acc: 0.1985 - val_loss: 0.9794 - val_acc: 0.2665
Epoch 14/25
144/144 [==============================] - 3s 21ms/step - loss: 0.4913 - acc: 0.3148 - val_loss: 0.9941 - val_acc: 0.6667
Epoch 15/25
144/144 [==============================] - 3s 23ms/step - loss: 0.4962 - acc: 0.7961 - val_loss: 0.9698 - val_acc: 0.5936
Epoch 16/25
144/144 [==============================] - 3s 23ms/

Epoch 14/25
144/144 [==============================] - 3s 20ms/step - loss: 0.3939 - acc: 0.1530 - val_loss: 0.9382 - val_acc: 0.1522
Epoch 15/25
144/144 [==============================] - 3s 19ms/step - loss: 0.4010 - acc: 0.1488 - val_loss: 0.9310 - val_acc: 0.4257
Epoch 16/25
144/144 [==============================] - 3s 18ms/step - loss: 0.3940 - acc: 0.4515 - val_loss: 0.9386 - val_acc: 0.1699
Epoch 17/25
144/144 [==============================] - 3s 19ms/step - loss: 0.3941 - acc: 0.1642 - val_loss: 0.9393 - val_acc: 0.1552
Epoch 18/25
144/144 [==============================] - 3s 20ms/step - loss: 0.3932 - acc: 0.1508 - val_loss: 0.9383 - val_acc: 0.1617
Epoch 19/25
144/144 [==============================] - 3s 18ms/step - loss: 0.3824 - acc: 0.1646 - val_loss: 0.9580 - val_acc: 0.1552
Epoch 20/25
144/144 [==============================] - 3s 18ms/step - loss: 0.3927 - acc: 0.1542 - val_loss: 0.9352 - val_acc: 0.1672
Epoch 21/25
144/144 [==============================] - 3s 18ms

144/144 [==============================] - 3s 18ms/step - loss: 0.3023 - acc: 0.1935 - val_loss: 0.9667 - val_acc: 0.1762
Epoch 20/25
144/144 [==============================] - 3s 18ms/step - loss: 0.2968 - acc: 0.2063 - val_loss: 0.9644 - val_acc: 0.1912
Epoch 21/25
144/144 [==============================] - 3s 18ms/step - loss: 0.2966 - acc: 0.2232 - val_loss: 0.9849 - val_acc: 0.1554
Epoch 22/25
144/144 [==============================] - 3s 18ms/step - loss: 0.2983 - acc: 0.1862 - val_loss: 0.9656 - val_acc: 0.1779
Epoch 23/25
144/144 [==============================] - 3s 18ms/step - loss: 0.2958 - acc: 0.2034 - val_loss: 0.9655 - val_acc: 0.1767
Epoch 24/25
144/144 [==============================] - 3s 18ms/step - loss: 0.2856 - acc: 0.2005 - val_loss: 0.9594 - val_acc: 0.1792
Epoch 25/25
144/144 [==============================] - 3s 18ms/step - loss: 0.2915 - acc: 0.2127 - val_loss: 0.9808 - val_acc: 0.1597
--------------------------------------------------
It's the third biggest 

144/144 [==============================] - 3s 21ms/step - loss: 0.2001 - acc: 0.2289 - val_loss: 1.0843 - val_acc: 0.1634
Epoch 25/25
144/144 [==============================] - 3s 22ms/step - loss: 0.1988 - acc: 0.2376 - val_loss: 1.0895 - val_acc: 0.1692
--------------------------------------------------
He has a lot of money. அவள் அவனுக்கு நிரும் செய்திற்குக்கே இருக்கின்

I'm short of money. அவள் அவனுக்கு நிரும் செய்திற்குக்கே இருக்கின்

It's been a long time since I've heard anyone use that word. அவள் அவனுக்கு நிரும் செய்திற்குக்கே இருக்கின்

The price of eggs is going up. அவள் அவனுக்கு நிரும் செய்திற்குக்கே இருக்கின்

Raise your hand. அவள் அவனுக்கு நிரும் செய்திற்குக்கே இருக்கின்

--------------------------------------------------
Iteration: 22
Train on 144 samples, validate on 36 samples
Epoch 1/25
144/144 [==============================] - 3s 19ms/step - loss: 0.2009 - acc: 0.2458 - val_loss: 1.1068 - val_acc: 0.1599
Epoch 2/25
144/144 [==============================] - 3s 20ms/s